In [ ]:
#!/user/bin/env python
#--------------------
#@Time  : 2020/4/14 18:00
#@Author: ZHANG YINGJIE
#@File  : utils.py
import math
import random

import cv2 as cv
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt


def readTrainData(img_path, gt_path, crop_size=256, scale=4, knn_phase=True, k=2, min_head_size=16, max_head_size=200):
    """
    read train data
    :param img_path: image path
    :param gt_path: the label(ground truth) path, .mat file path
    :param crop_size: crop size
    :param knn_phase: 布尔型，是否使用几何自适应高斯核
    :param k: integer, k近邻数
    :param min_head_size: 原始行人头部尺寸的缩放最小值
    :param max_head_size: 原始行人头部尺寸的缩放最大值
    :return: 网络输入图像，以比例尺密度图作为网络的地面真值密度图，地面真值人群计数值
    """
    # read image
    crowd_img = cv.imread(img_path)
    # read .mat file
    label_data = loadmat(gt_path)
    # dict_keys(['__header__', '__version__', '__globals__', 'image_info'])
    # # __header__
    # # b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Fri Oct 23 13:18:27 2015'
    # # __version__
    # # 1.0
    # # __globals__
    # # []
    # # image_info
    # # [[array([[(array([[ 29.6225116 , 472.92022152],
    # #        [ 54.35533603, 454.96602305],
    # #        [ 51.79045053, 460.46220626],
    # #        ...,
    # #        [597.89732076, 688.27900015],
    # #        [965.77518336, 638.44693908],
    # #        [166.9965574 , 628.1873971 ]]), array([[1546]], dtype=uint16))]],
    # #       dtype=[('location', 'O'), ('number', 'O')])]]
    points = label_data['image_info'][0][0]['location'][0][0]
    cropped_crowd_img, cropped_points, cropped_crowd_count = getCroppedCrowdImg(crowd_img, points, crop_size)

    cropped_scaled_crowd_img, cropped_scaled_points = getScaledCrowdImg(cropped_crowd_img, cropped_points, scale)
    cropped_scaled_crowd_img_size = [cropped_scaled_crowd_img.shape[0], cropped_scaled_crowd_img.shape[1]]
    cropped_crowd_img_size = [cropped_crowd_img.shape[0], cropped_crowd_img.shape[1]]
    scaled_min_head_size = min_head_size / scale
    scaled_max_head_size = max_head_size / scale

    # after cropped
    density_map = getDensityMap(cropped_crowd_img_size, cropped_points,
                                knn_phase, k, min_head_size, max_head_size)
    scaled_density_map = getDensityMap(cropped_scaled_crowd_img_size, cropped_scaled_points,
                                       knn_phase, k, scaled_min_head_size, scaled_max_head_size)

    cropped_crowd_img = cropped_crowd_img.reshape((1, cropped_crowd_img.shape[0], cropped_crowd_img.shape[1], cropped_crowd_img.shape[2]))
    cropped_crowd_count = np.asarray(cropped_crowd_count).reshape((1,1))
    cropped_density_map = density_map.reshape((1, density_map.shape[0], density_map.shape[1], 1))
    cropped_scaled_density_map = scaled_density_map.reshape((1,scaled_density_map.shape[0],scaled_density_map.shape[1],1))

    return cropped_crowd_img, cropped_density_map, cropped_crowd_count, cropped_scaled_density_map


def readTestData(img_path, gt_path, scale=4, deconv_is_used=False, knn_phase=True, k=2, min_head_size=16, max_head_size=200):
    """
    read test data
    """
    crowd_img = cv.imread(img_path)
    label_data = loadmat(gt_path)
    points = label_data['image_info'][0][0]['location'][0][0]
    crowd_count = label_data['image_info'][0][0]['number'][0][0]
    h, w = crowd_img.shape[0], crowd_img.shape[1]

    if deconv_is_used:
        h_ = h - (h // scale) % 2
        rh = h_ / h
        w_ = w - (w // scale) % 2
        rw = w_ / w
        crowd_img = cv.resize(crowd_img, (w_, h_))
        points[:, 1] = points[:, 1] * rh
        points[:, 0] = points[:, 0] * rw

    scaled_crowd_img, scaled_points = getScaledCrowdImg(crowd_img, points, scale)
    scaled_img_size = [scaled_crowd_img.shape[0], scaled_crowd_img.shape[1]]
    img_size = [crowd_img.shape[0], crowd_img.shape[1]]
    scaled_min_head_size = min_head_size / scale
    scaled_max_head_size = max_head_size / scale

    density_map = getDensityMap(img_size, points, knn_phase, k, min_head_size, max_head_size)
    scaled_density_map = getDensityMap(scaled_img_size, points, knn_phase, k, scaled_min_head_size, scaled_max_head_size)
    crowd_img = crowd_img.reshape((1, crowd_img.shape[0], crowd_img.shape[1], crowd_img.shape[2])) # "NHWC"
    crowd_count = np.asarray(crowd_count).reshape((1,1))
    density_map = density_map.reshape((1, density_map.shape[0], density_map.shape[1], 1))
    scaled_density_map = scaled_density_map.reshape((1, scaled_density_map.shape[0], scaled_density_map.shape[1], 1))

    return crowd_img, density_map, crowd_count, scaled_density_map

def getCroppedCrowdImg(crowd_img, point_positions, crop_size):
    """
    Crops a sub-crowd image randomly
    :param crowd_img: 原始图像，shape is [h,w,channel]
    :param point_positions: 人头点的位置集
    :param crop_size: 裁剪大小
    :return: 裁剪后图像，裁剪后人头位置点集，裁剪后图像的人头数
    """
    high, weight = crowd_img.shape[0], crowd_img.shape[1]

    # h&w < crop_size, reduce crop_size
    if high < crop_size or weight < crop_size:
        crop_size = crop_size // 2

    # random crop
    x1 = random.randint(0, high - crop_size)
    y1 = random.randint(0, weight - crop_size)
    x2 = x1 + crop_size
    y2 = y1 + crop_size

    cropped_crowd_img = crowd_img[x1:x2, y1:y2, ...]

    cropped_point_positions = []
    for i in range(len(point_positions)):
        if x1 <= point_positions[i,1] <= x2 and y1 <= point_positions[i,0] <=y2:
            point_positions[i,0] = point_positions[i,0] - y1
            point_positions[i,1] = point_positions[i,1] - x1
            cropped_point_positions.append(point_positions[i]) # new location after crop
    cropped_point_positions = np.asarray(cropped_point_positions)
    cropped_crowd_count = len(cropped_point_positions)

    return cropped_crowd_img, cropped_point_positions, cropped_crowd_count


def getScaledCrowdImg(crowd_img, point_positions, scale):
    """
    Gets scaled crowd img and scaled points
    :param crowd_img: 输入图像
    :param point_positions: 人头点集
    :param scale: 比例因子
    :return: 按比例因子调整后的图像和点集
    """
    high, weight = crowd_img.shape[0], crowd_img.shape[1]
    scaled_crowd_img = cv.resize(crowd_img, (weight // scale, high // scale))
    for i in range(len(point_positions)):
        point_positions[i] = point_positions[i] / scale

    return scaled_crowd_img, point_positions


# def getDensityMap(scaled_crowd_img_size, scaled_points, knn_phase, k, scaled_min_head_size, scaled_max_head_size):
#     """
#     生成相应的地面真实密度图
#     :param scaled_crowd_img_size: 密度图大小
#     :param scaled_points: 位置点集
#     :param knn_phase: 布尔型，是否使用几何自适应高斯核
#     :param k: 近邻数
#     :param scaled_min_head_size: 原始行人头部尺寸的缩放最小值
#     :param scaled_max_head_size: 原始行人头部尺寸的缩放最大值
#     :return: density map
#     """
#     h, w = scaled_crowd_img_size[0], scaled_crowd_img_size[1]
#     density_map = np.zeros((h,w))
#
#     num = len(scaled_points)
#     if num == 0:
#         return density_map
#     for i in range(num):
#         # point position is [oy,ox], represents as [x,y]
#         x = min(h, max(0, abs(int(math.floor(scaled_points[i,1])))))
#         y = min(w, max(0, abs(int(math.floor(scaled_points[i,0])))))
#
#         position = [x, y]
#
#         sigma = 1.5
#         beta = 0.3
#         ksize = 25
#         if knn_phase:
#             avg_distance = getAvgDistance(position, scaled_points, k)
#             avg_distance = max(min(avg_distance, scaled_max_head_size), scaled_min_head_size)
#             sigma = beta * avg_distance
#             ksize = 1.0 * avg_distance
#
#         # 边缘处理
#         x1 = x - int(math.floor(ksize / 2))
#         y1 = y - int(math.floor(ksize / 2))
#         x2 = x + int(math.ceil(ksize / 2))
#         y2 = y + int(math.ceil(ksize / 2))
#
#         if x1 < 0 or y1 < 0 or x2 > h or y2 > w:
#             x1 = max(0, x1)
#             y1 = max(0, y1)
#             x2 = min(h, x2)
#             y2 = min(w, y2)
#             tmp = min((x2 - x1), (y2 - y1))
#             ksize = min(tmp, ksize)
#
#         ksize = int(math.floor(ksize / 2))
#         H = fspecial(ksize, sigma) # 2D Gaussian kernel
#         density_map[x1:x1+ksize, y1:y1+ksize] = density_map[x1:x1+ksize, y1:y1+ksize] + H
#
#     return np.asarray(density_map)

def getDensityMap(crowd_img_size, points, knn_phase, k, min_head_size, max_head_size):
    """
    生成相应的地面真实密度图
    :param crowd_img_size: 密度图大小
    :param points: 位置点集
    :param knn_phase: 布尔型，是否使用几何自适应高斯核
    :param k: 近邻数
    :param min_head_size: 原始行人头部尺寸的缩放最小值
    :param max_head_size: 原始行人头部尺寸的缩放最大值
    :return: density map
    """
    h, w = crowd_img_size[0], crowd_img_size[1]
    density_map = np.zeros((h,w))

    num = len(points)
    if num == 0:
        return density_map
    for i in range(num):
        # point position is [oy,ox], represents as [x,y]
        x = min(h, max(0, abs(int(math.floor(points[i,1])))))
        y = min(w, max(0, abs(int(math.floor(points[i,0])))))

        position = [x, y]

        sigma = 1.5
        beta = 0.3
        ksize = 25
        if knn_phase:
            avg_distance = getAvgDistance(position, points, k)
            avg_distance = max(min(avg_distance, max_head_size), min_head_size)
            sigma = beta * avg_distance
            ksize = 1.0 * avg_distance

        # 边缘处理
        x1 = x - int(math.floor(ksize / 2))
        y1 = y - int(math.floor(ksize / 2))
        x2 = x + int(math.ceil(ksize / 2))
        y2 = y + int(math.ceil(ksize / 2))

        if x1 < 0 or y1 < 0 or x2 > h or y2 > w:
            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(h, x2)
            y2 = min(w, y2)
            tmp = min((x2 - x1), (y2 - y1))
            ksize = min(tmp, ksize)

        ksize = int(math.floor(ksize / 2))
        H = fspecial(ksize, sigma) # 2D Gaussian kernel
        density_map[x1:x1+ksize, y1:y1+ksize] = density_map[x1:x1+ksize, y1:y1+ksize] + H

    return np.asarray(density_map)

def getAvgDistance(position, points, k):
    """
    k近邻平均距离
    :param position: 当前点位置
    :param points: 位置点集
    :param k: 近邻数
    :return: avg distance
    """
    num = len(points)
    if num == 1:
        return 1.0
    elif num <= k:
        k = num - 1

    euclidean_distance = np.zeros((num,1))
    for i in range(num):
        x = points[i,1]
        y = points[i,0]
        euclidean_distance[i,0] = math.sqrt(math.pow(position[1] - x, 2) + math.pow(position[0] - y, 2))

    euclidean_distance[:,0] = np.sort(euclidean_distance[:,0])
    avg_distance = euclidean_distance[1:k+1, 0].sum() / k
    return avg_distance


def fspecial(ksize, sigma):
    """
    Generates 2d Gaussian kernel
    :param ksize: an integer, represents the size of Gaussian kernel
    :param sigma: a float, represents standard variance of Gaussian kernel
    :return: 2d Gaussian kernel, the shape is [ksize, ksize]
    """
    # [left, right)
    left = -ksize / 2 + 0.5
    right = ksize / 2 + 0.5
    x, y = np.mgrid[left:right, left:right]
    # generate 2d Gaussian Kernel by normalization
    gaussian_kernel = np.exp(-(np.square(x) + np.square(y)) / (2 * np.power(sigma, 2))) / (2 * np.power(sigma, 2)).sum()
    sum = gaussian_kernel.sum()
    normalized_gaussian_kernel = gaussian_kernel / sum

    return normalized_gaussian_kernel


def showDensityMap(density_map):
    """
    show the density map
    :param density_map: density map,shape [h,w]
    :return:
    """
    plt.imshow(density_map, cmap='jet')
    plt.show()


